In [65]:
using Pkg; Pkg.activate("../.."); Pkg.instantiate()
using RxInfer,Distributions,Random,LinearAlgebra,OhMyREPL, ReactiveMP
enable_autocomplete_brackets(false);colorscheme!("GruvboxDark");

# TODO: Structure is correct now but the results are fucked
# include("GFECategorical.jl")
include("../GFECategorical2.jl")
include("../helpers.jl")

  Activating project at `~/.julia/dev/EpistemicMessagePassing`


constructABCD (generic function with 1 method)

In [101]:
@rule Transition(:in, Marginalisation) (q_out::Any, q_a::Any) = begin
    a = clamp.(exp.(mean(log, q_a)' * probvec(q_out)), tiny, Inf)
    return Categorical(a ./ sum(a))
end

@model function t_maze(A,B,C,T, pipeline = nothing)

    D_0 = datavar(Vector{Float64})
    z_0 ~ Categorical(D_0)
    # We use datavar here since x~ Pointmass is not a thing
    x = datavar(Vector{Float64},T)
    z = randomvar(T)
    w = randomvar(T)

    # Requires changes in the ReactiveMP core, `@meta` does not support pipelines (yet)
    pipeline = something(pipeline, ReactiveMP.DefaultFunctionalDependencies())

    z_prev = z_0

    for t in 1:T
        z[t] ~ Transition(z_prev, B[t])
        w[t] ~ Transition(z[t], A) where { pipeline = pipeline }
        w[t] ~ Categorical(x[t])
        z_prev = z[t]
    end
end;

# Edge Constraints
@constraints [ warn = false ] function gfeconstraints()
    q(x, z, w, A) = q(x)q(w)q(z)q(A)
    q(w) :: EpistemicProduct
end

# Node constraints
@meta function gfemeta()
    Transition(z, w) -> EpistemicMeta()
    Categorical(x, w) -> EpistemicMeta()
end

@constraints [ warn = false ] function bfeconstraints()
    q(x, z, w, A) = q(x)q(w)q(z)q(A)
end

# @meta function t_maze_bfe()

# end

bfeconstraints (generic function with 1 method)

In [102]:
gfepipeline = GFEPipeline((2,))

A,B,C,D = constructABCD(0.9,[2.,2.],2);

In [107]:
gfe_setup = (
    pipeline = gfepipeline,
    constraints = gfeconstraints(),
    meta = gfemeta()
)

bfe_setup = (
    pipeline = nothing,
    constraints = bfeconstraints(),
    meta = nothing
)

current_setup = gfe_setup

(pipeline = GFEPipeline{Tuple{Int64}}((2,)), constraints = Constraints:
  marginals form:
    q(w) :: EpistemicProduct() [ prod_constraint = ProdGeneric(fallback = ProdAnalytical()) ]
  messages form:
  factorisation:
    q(x, z, w, A) = q(x)q(w)q(z)q(A)
Options:
  warn = false
, meta = Meta specification:
  Transition(z, w) -> EpistemicMeta()
  Categorical(x, w) -> EpistemicMeta()
Options:
  warn = true)

In [108]:
T = 2

initmarginals = (
                 z = [Categorical(fill(1/8,8)) for t in 1:T],
                );

initmessages = (
                 z = [Categorical(fill(1/8,8)) for t in 1:T],
             );


# TODO: Figure out why FE increases with number of iterations - but only for the best policy?
result = nothing
its = 50
F = zeros(4,4);
results = Dict()
for i in 1:4
    for j in 1:4
        Bs = (B[i],B[j])
        global result = inference(model = t_maze(A,Bs,C,T, current_setup[:pipeline]),
                           data= (D_0 = D, x = C),
                           initmarginals = initmarginals,
                           initmessages = initmessages,
                           constraints = current_setup[:constraints],
                           meta = current_setup[:meta],
                           free_energy=true,
                           # addons = (AddonMemory(),),
                           iterations = its)
       results[(i, j)] = result
        F[i,j] = result.free_energy[end]
    end
end

# q_w is always one iteration behind
# Wmarginal.q stays flat? Initial message towards GFEnode never gets updated

In [109]:
F

4×4 Matrix{Float64}:
 7.2803   6.22057  6.22057  6.58715
 6.22057  6.22057  6.22057  6.22057
 6.22057  6.22057  6.22057  6.22057
 6.58715  5.75922  5.75922  5.89401